# Project #2. Extented Kalman Filter. Localization

The project is denoted to nonlinear Kalman Filter version - so called [Extended Kalman Filter (EKF)](https://en.wikipedia.org/wiki/Extended_Kalman_filter).

EKF has various applications. One of them is Localization task.

Localization - robot state estimation through sensor fusion and prior map knoledge. It means that robot's pose is unknown, but the map is given. Using the knoledge of control signals:
$${u_0, ..., u_t}$$ and measurements: $$z_0, ..., z_t$$ current robot's state $x_t$ should be estimated.

Assuptions:
- feature-based map is known. Use several variants of maps
- features are destingwishable - i.e. correspondeces are known
- robot has a bicycle kinematics
- robot has a mathematical model of "absolute features sensor". This sensor can detect and distinguish all features around the robot at range of R = 10 meters
- robot has gps sensor. This sensor returns robot's position with some error
- map has size of 200 by 200 meters


## Workflow of the project:
- Implement classes and it's methods:

```Feature```, ```Map```, ```Feature_Sensor```, ```EKF_Localizer```, ```EKF_Experiment```
- Use bycicle model with PID controller from [project #1](http://localhost:8888/tree/Bicycle%20model.%20PID%20controller) to make robot move along some path on the map
- Tune up measurement noise covariance matrix
- Tune up motion noise covariance matrix in space of $$(v, w)$$ control signals, where $v$ - linear speed and $w$ - angular speed. The kinemmatical equations of bicycle model should be improved to take $(v,w)$ as a control inputs
- The state of the EKF filter should converge to ground truth position of the robot
- Plot graphs and animations
- Write small report
- EKF state vector should have dimension of **3**: $$\overline{x} = (x,y,\theta)$$

## Graphs to plot:
- plot map with it's features. Print all features's id's
- plot actual position of the robot and EKF estimation (covariance ellipsoid)
- plot robot's trajectory


## Write little report. Give answers on the following questions:
- Try several different maps. Are they all sutable for EKF tuning?
- What happens if robot doesn't move and observe only one feature? Two features? Three features?
- What further improvements can be done on EKF filter?



## My contacts
- Email: Panchenko@cognitive.ru
- Telegram: +7-903-270-78-29


In [1]:
class Feature:
    def __init__(self, x, y, s):
        #
        # TODO: write some code here for Feature class initialization
        #
        pass

In [3]:
class Map:
    features = [] # put map features in a list
    

In [6]:
class FeatureSensor:
    R = 5
    def __init__(self, R, Map):
        pass
    def getMeasurements(self, robot_pose):
        #
        # TODO: returns list of measurements with reference to current robot pose and sensor range
        #
        pass

$x_{t+1} = g(u, x_t) + \delta_t$, where $\delta_t$ - is a noise inducted from $(u,v)$ control space.

G - Jacobian of function g:

$$G = \dfrac{dg(u,x)}{dx}$$

$z = (r,\phi)$ - where $r$ - is distance to feature, $\phi$ - the bearing to feature.

$z = \left(\sqrt{(x - x_i)^2 + (y - y_i)^2}, arctan\left(\dfrac{y - y_i}{x - x_i}\right) - \theta\right) = h(x)$

V - Jacobian of function h:

$$V = \dfrac{dh(x)}{dx}$$


In [11]:
class EKF_Localizer:
    def __init__(self):
        pass
    def predict(control):
        #
        # TODO: do prediction step
        #
        pass
    def update(measurements):
        #
        # TODO: do update step
        #
        pass
    def getG():
        #
        # TODO: computer Jacobian of nonlinear function g(u,x) with respect to state x
        #
        pass
    def getV():
        #
        # TODO: compute Jacobian of nonlinear function g(u,x) with respect to control u
        #
        pass
    def getH():
        pass
        
    

## Project report:

Put your report here. Feel free to use pictures, graphs, animations.

